In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 8s (33.2 kB/s)
Reading package li

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-01 23:48:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.00MB/s    in 0.2s    

2022-05-01 23:48:38 (6.00 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

#df.show()
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [5]:
from pyspark.sql.types import IntegerType

In [6]:
df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))
df = df.withColumn("product_parent", df["product_parent"].cast(IntegerType()))
df = df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df = df.withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))
df = df.withColumn("total_votes", df["total_votes"].cast(IntegerType()))


### Create DataFrames to match tables

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

In [8]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [29]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"*": "count"}).withColumnRenamed("count(1)", "customer_count")
customers_df.count()
customers_df.show(10)





+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35669025|             1|
|   48198949|            30|
|   43355824|             6|
|   33014261|             6|
|   23493243|             1|
|   30717305|             4|
|   15714077|             2|
|    7854719|             1|
|   12761428|             2|
|   14127895|             1|
+-----------+--------------+
only showing top 10 rows



In [10]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
#products_df = products_df.drop_duplicates(["product_id"])
#print(products_df.count())
products_df.show(10)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0092702VE|Abu Garcia Revo S...|
|B00HZ9WTVC|Lee 90355 Cutter ...|
|B00WS4FRQ8|BEEWARE Targets -...|
|B00AKJVYYC|NEW Callaway RAZR...|
|B00DIPSOM0|GoSports Ladder T...|
|B0017LEUCU|Body Sport Exerci...|
|B00B59OMS6|Swiftwick SUSTAIN...|
|B00KC5SYLU|EliteTek RG-14 Fo...|
|B000GKUAB8|Multisport Helmet...|
|B001P80Z10|Izzo Golf Mini Mo...|
+----------+--------------------+
only showing top 10 rows



In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
print(review_id_df.count())
review_id_df.show(5)

4850360
+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
print(vine_df.count())
vine_df.show(10)

4850360
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|  

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [22]:
# Configure settings for RDS
import getpass
password = getpass.getpass("bonito81")
mode = "append"
jdbc_url="jdbc:postgresql://mypostgres2db.cmxry9tohzk8.us-west-1.rds.amazonaws.com/my_challenge_16_db"
config = {"user":"root", 
          "password": "bonito81*", 
          "driver":"org.postgresql.Driver"}


bonito81··········


In [23]:
short_review = review_id_df.limit(50)

In [24]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [25]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [26]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [27]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)